In [2]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chardet # To provide a best estimate of the encoding that was used in the text data
import io # For string operations
%matplotlib inline

# Libraries for data preparation and model building
#from imblearn.over_sampling import RandomOverSampler
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, TweetTokenizer, TreebankWordTokenizer
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import math
import re
from sklearn.utils import resample
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn import feature_selection
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif, mutual_info_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, log_loss
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE,SMOTENC
from imblearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dawie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dawie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [45]:
csv_file = r'G:\My Drive\Professionele ontwikkeling\Data Science\Explore Data Science Course\Sprint 6_Advanced Classification\Predict\advanced-classification-predict\data\train.csv'
num_records_to_load = 3000  # Adjust this number as needed
np.random.seed(42) 
pd.read_csv(csv_file).shape[0] - num_records_to_load
df_train = pd.read_csv(csv_file, skiprows=lambda i: i > 0 and np.random.rand() > num_records_to_load / (i + 1), nrows=num_records_to_load)
#df_train = pd.read_csv(csv_file)
pd.set_option('display.max_colwidth', None)
results_counter = 1
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  3000 non-null   int64 
 1   message    3000 non-null   object
 2   tweetid    3000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 70.4+ KB


In [8]:
# Step 1: Split into train / test
X_train, X_test, y_train, y_test = train_test_split(df_train['message'], df_train['sentiment'], test_size=0.2, random_state=42)


In [31]:
X_train.info()

 1    7205
 2    3111
 0    2040
-1    1090
Name: sentiment, dtype: int64

https://www.andrewvillazon.com/custom-scikit-learn-transformers/

SIEN OOK PDF IN DOWNLOADS: How To Write Clean And Scalable Code With Custom Transformers & Sklearn Pipelines _ by Beng Chew _ Medium


In [10]:
class SampleSelector(BaseEstimator, TransformerMixin):
    def __init__(self, sample_percentage=1.0, stratify='y', random_state=42):
        self.sample_percentage = sample_percentage
        self.stratify = stratify
        self.random_state = random_state

    def fit(self, X, y):
        return self

    def transform(self, X, y=None):
        # Set random seed for reproducibility
        np.random.seed(self.random_state)
        if self.sample_percentage != 1.0:
            print(X.shape)
            print(y.shape)
            print(f'y:')
            print(y.iloc[0])
            # Determine the unique classes in y and their frequencies
            unique_classes, class_counts = np.unique(y, return_counts=True)
            # Determine the number of samples to select for each class
            num_samples_per_class = (class_counts * self.sample_percentage).astype(int)
            # Initialize an empty list to store the sampled indices
            print(unique_classes)
            print(class_counts)
            sampled_indices = []

            # Iterate over each unique class
            for cls, num_samples in zip(unique_classes, num_samples_per_class):
                # Get the indices of samples belonging to the current class
                class_indices = np.where(y == cls)[0]
                # Randomly select samples from the current class
                sampled_indices.extend(np.random.choice(class_indices, size=num_samples, replace=False))

            print(sampled_indices)
            X_sampled = X.iloc[sampled_indices]
            y_sampled = y.iloc[sampled_indices]

        else:
            X_sampled = X
            y_sampled = y

        # Return the randomly sampled subset

        print(type(X_sampled))
        print(f'X_sampled[0]: {X_sampled.iloc[0]}')
        print(f'y_sampled[0]: {y_sampled.iloc[0]}')
        return X_sampled, y_sampled

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        X_sampled = self.transform(X, y)
        return X_sampled, y_sampled

In [11]:
class NoiseRemover(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y):
        return self

    def transform(self, X, y=None):
        pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'   # Find all hyperlinks
        subs_url = r''
        X_transformed = pd.Series(X).replace(to_replace = pattern_url, value = subs_url, regex = True)
        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [12]:
class EmoticonConverter(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y):
        return self

    def transform(self, X, y=None):
        emoticon_dictionary = {':\)': 'smiley_face_emoticon',
                               ':\(': 'frowning_face_emoticon',
                               ':D': 'grinning_face_emoticon',
                               ':P': 'sticking_out_tongue_emoticon',
                               ';\)': 'winking_face_emoticon',
                               ':o': 'surprised_face_emoticon',
                               ':\|': 'neutral_face_emoticon',
                               ':\'\)': 'tears_of_joy_emoticon',
                               ':\'\(': 'crying_face_emoticon'}
        X_transformed = X.replace(emoticon_dictionary, regex=True)
        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [13]:
class PunctuationRemover(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y):
        return self

    def transform(self, X, y=None):
        #print(X.iloc[1])
        def expand_contractions(text):
            #print(f'X type during function expand_contractions: {type(text)}')
            contractions = {"'t": " not","'s": " is","'re": " are","'ll": " will", "'m": " am"}
            pattern = re.compile(r"\b(" + "|".join(re.escape(key) for key in contractions.keys()) + r")\b")
            text = re.sub(r"n't\b", " not", text) # Replace "n't" with " not"
            text = pattern.sub(lambda match: contractions[match.group(0)], text) # Replace all other contractions except for "n't"
            return text

        def remove_punctuation(text):
            return ''.join([l for l in text if l not in string.punctuation])

        #print(f'X type before transformation: {type(X)}')
        #X_transformed = X.apply(expand_contractions)
        X_transformed = X.apply(lambda x: expand_contractions(x))
        #print(f'X_transformed type after expand function: {type(X_transformed)}')
        #X_transformed = X_transformed.apply(remove_punctuation)
        X_transformed = X_transformed.apply(lambda x: remove_punctuation(x))


        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [14]:
class Tokenizer(BaseEstimator, TransformerMixin):
    def __init__(self, type='TweetTokenizer'):
        self.type = type

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if self.type == 'TweetTokenizer':
            tokenizer = TweetTokenizer()
        else:
            tokenizer = TreebankWordTokenizer()
        X_transformed = X.apply(tokenizer.tokenize)
        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [15]:
class StopwordRemover(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        stop_words = set(stopwords.words('english'))
        # Remove stopwords using a vectorized operation
        X_transformed = X.apply(lambda tokens: [t for t in tokens if t.lower() not in stop_words])
        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [16]:
class Lemmatizer(BaseEstimator, TransformerMixin):
    def __init__(self, pos='v'):
        self.pos = pos

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        lemmatizer = WordNetLemmatizer()
        X_transformed = X.apply(lambda tokens: [lemmatizer.lemmatize(word, pos=self.pos) for word in tokens])
        return X_transformed

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [17]:
class Vectorize(BaseEstimator, TransformerMixin):
    def __init__(self, type='tfidf', max_df=1, min_df=1, ngram_range=(1,2), max_features=None):
        self.type = type
        self.max_df = max_df
        self.min_df = min_df
        self.ngram_range = ngram_range
        self.max_features = max_features

        if self.type == 'count':
            self.vectorizer = CountVectorizer(max_features=self.max_features, lowercase=True,
                                              max_df=self.max_df, min_df=self.min_df,
                                              ngram_range=self.ngram_range)
        elif self.type == 'tfidf':
            self.vectorizer = TfidfVectorizer(max_features=self.max_features, lowercase=True,
                                              max_df=self.max_df, min_df=self.min_df,
                                              ngram_range=self.ngram_range)
        else:
            raise ValueError("Invalid vectorizer type. Choose either 'count' or 'tfidf'")

    def fit(self, X, y=None):
        X_joined = [' '.join(tokens) for tokens in X]
        self.vectorizer.fit(X_joined)
        return self

    def transform(self, X, y=None):
        X_joined = [' '.join(tokens) for tokens in X]
        return self.vectorizer.transform(X_joined)

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [18]:
class Resampler(BaseEstimator, TransformerMixin):
    def __init__(self, technique='balanced', random_state=42):
        self.technique = technique
        self.random_state = random_state

    def fit(self, X, y):
        return self

    def transform(self, X, y=None):
        if self.technique == 'balanced':
            class_labels, counts = np.unique(y, return_counts=True)
            n_classes = class_labels.shape[0]
            balanced_freq = X.shape[0] / n_classes  # Get the number of samples from the shape of X
            X_resampled, y_resampled = [], []
            for class_label, count in zip(class_labels, counts):
                indices = np.atleast_1d(np.where(y == class_label)[0])
                if count > balanced_freq:
                    resampled_indices = resample(indices, replace=False, n_samples=int(balanced_freq), random_state=self.random_state)
                elif count < balanced_freq:
                    resampled_indices = resample(indices, replace=True, n_samples=int(balanced_freq), random_state=self.random_state)
                else:
                    resampled_indices = indices
                if len(resampled_indices) > 0:
                    resampled_X = X[resampled_indices]
                    X_resampled.append(resampled_X)
                    y_resampled.extend([class_label] * resampled_X.shape[0])

            if X_resampled:
                X_resampled = vstack(X_resampled)
            else:
                X_resampled = csr_matrix((0, X.shape[1]))  # Create an empty sparse matrix
            y_resampled = np.array(y_resampled)

        elif self.technique == 'smote':
            sampler = SMOTE(random_state=self.random_state)
            X_dense = X.toarray()
            X_resampled, y_resampled = sampler.fit_resample(X_dense, y)
            X_resampled = csr_matrix(X_resampled)

        else:
            raise ValueError("Invalid resampling technique.  Choose either 'random' or 'smote'")

        return X_resampled, y_resampled

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [19]:
class Scaler(BaseEstimator, TransformerMixin):
    def __init__(self, type='robust'):
        self.type = type

        if self.type == 'robust':
            self.scaler = RobustScaler(with_centering=False)
        elif self.type == 'minmax':
            self.scaler = MinMaxScaler(with_centering=False)
        elif self.type == 'maxabs':
            self.scaler = MaxAbsScaler(with_centering=False)
        else:
            raise ValueError("Invalid scaler type. Choose between 'robust', 'minmax' or 'maxabs'.")

    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self

    def transform(self, X, y=None):
        return self.scaler.transform(X)

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [20]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, type='anova_f', percentile=50):
        self.type = type
        self.percentile = percentile

        if self.type == 'mutualinfo':
            self.selector = SelectPercentile(score_func=mutual_info_classif, percentile=percentile)
        elif self.type == 'anova_f':
            self.selector = SelectPercentile(score_func=f_classif, percentile=percentile)
        else:
            raise ValueError("Invalid selector type. Choose between 'mutualinfo' or 'anova_f'.")

    def fit(self, X, y=None):
        print(y.head())
        self.selector.fit(X, y)
        return self

    def transform(self, X, y=None):
        return self.selector.transform(X)

    def fit_transform(self, X, y=None):
        self.selector.fit(X, y)
        return self.selector.transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train['message'], df_train['sentiment'], test_size=0.15, random_state=42)
# Defining preprocessing steps
preprocessing_steps = [
                       ('noise_removal', NoiseRemover()),
                       ('emoticon_convertion', EmoticonConverter()),
                       ('punctuation_removal', PunctuationRemover()),
                       ('tokenization', Tokenizer()),
                       ('stopword_removal', StopwordRemover()),
                       ('lemmatization', Lemmatizer()),
                       ('vectorization', Vectorize()),
                       ('smote', SMOTE()),
                       ('scaler', Scaler()),
                       ('feature_selection', FeatureSelector())
                       ]

# Defining the models
models =               {
                        'Logistic Regression': LogisticRegression(),
                        'Support Vector Machine': SVC(),
                        'Multinomial Naive Bayes': MultinomialNB(),
                        'Decision Tree': DecisionTreeClassifier(),
                        'K-Nearest Neighbors': KNeighborsClassifier(),
                        'Random Forest': RandomForestClassifier(),
                       }

# Create the pipelines
pipeline =              Pipeline(preprocessing_steps  + [('model', LogisticRegression())])



# Define parameter grid for GridSearchCV
param_grid = {'tokenization__type': ['TweetTokenizer'],# 'TreebankWordTokenizer'],
              'vectorization__type': ['tfidf','count'],
              'vectorization__max_df': [0.25, 0.5],#,0.75],#,0.75],
              'vectorization__min_df': [1],#,2],#10],
              'vectorization__ngram_range': [(1,2), (1,5)],#,(1,3)],
              'vectorization__max_features': [200, 300, 500],#, 1000],
              'scaler__type': ['robust'],#'minmax','maxabs'],
              'feature_selection__type': ['anova_f'],
              'feature_selection__percentile': [25,50],#,75],

            # Hyperparameters for the model

              #'model__n_neighbors': [3],
              #'model__weights': ['uniform', 'distance'],
              #'model__metric': ['euclidean', 'manhattan'],

}

# Create GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro', error_score='raise', verbose=2)
grid_search.fit(X_train, y_train)
df_gridsearch = pd.DataFrame(grid_search.cv_results_)

print(df_gridsearch)
print(grid_search.get_params)
# print to Excel
'''
# Evaluate the pipeline
pipeline.fit(X_train, y_train)
accuracy = pipeline.score(X_test, y_test)
print("Accuracy:", accuracy)

model_stats = pipeline.named_steps['model']

# Obtain the number of features used by the model
attributes = model_stats.__dict__

# Print all attributes
for attr, value in attributes.items():
    print(attr, ":", value)
'''

In [ ]:
# MODEL COMPARISON

preprocessing_steps = [
                       ('noise_removal', NoiseRemover()),
                       ('emoticon_convertion', EmoticonConverter()),
                       ('punctuation_removal', PunctuationRemover()),
                       ('tokenization', Tokenizer(type='tfidf', max_df=0.25, min_df=2, ngram_range=(1,7), max_features=100)),
                       ('stopword_removal', StopwordRemover()),
                       ('lemmatization', Lemmatizer()),
                       ('vectorization', Vectorize()),
                       ('smote', SMOTE()),
                       ('scaler', Scaler(type='robust')),
                       ('feature_selection', FeatureSelector(type='anova_f',percentile=25))
                       ]

# Create the pipelines
pipeline = Pipeline(preprocessing_steps)
pipeline.fit(X_train, y_train)


names = [
         'Logistic Regression',
         'Nearest Neighbors',
         'Linear SVM',
         'RBF SVM',
         'Decision Tree',
         'Random Forest',
         'AdaBoost',
         'Gaussian Naive Bayes',
         #'Multinomial Naive Bayes',
         #'XGBoost'
         ]

classifiers = [
               LogisticRegression(max_iter=1000),
               KNeighborsClassifier(n_neighbors=3), #algorithm='ball_tree', leaf_size=30, metric='minkowski',
                                    # p=2, weights='distance'),
               SVC(kernel="linear", C=0.025),
               SVC(gamma=1, C=1),
               DecisionTreeClassifier(max_depth=5),
               RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, random_state=42),
               AdaBoostClassifier(random_state=42, n_estimators=200),
               GaussianNB(),
               #MultinomialNB(alpha=0.05),
               #xgb.XGBClassifier(objective='multi:softmax', max_depth=10, num_class=len(set(y_train)))
              ]
'''
param_grids = [
               {}, #Logistic Regression
               {'n_neighbors': [3], 'metric': ['euclidean'],#,'minkowski',  'manhattan', 'hamming', 'cosine', 'correlation','jaccard'],
                'weights': ['distance']}#'uniform', ], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
               # 'leaf_size': [30,50], 'p': [1, 2]}, #Nearest Neighbors
               #{}, #Linear SVM
               #{'kernel': ['rbf'],'gamma': (0.001,0.01,1),'C': (0.1,1,10), 'class_weight': [None, 'balanced'],'probability': [True, False],
               #'shrinking': [True, False], 'tol': [1e-3, 1e-4] } #RBF SVM
               #{}, #Decision Tree
               #{'max_depth': [2,5,10], 'n_estimators': [50,100,250], 'random_state': [42]}, #Random Forest
               #{'random_state': [42], 'n_estimators': [50,100,200], 'learning_rate': [0.1, 1, 2]}, #Adaboost
               #{}, #Gaussian Naive Bayes
               #{'alpha': [0.05, 0.1, 0.5, 1]}, #Multinomial Naive Bayes
               #{}, #XGBoost
              ]
'''
# Train all the models

results = []
models = {}
confusion = {}
class_report = {}

for name, clf in zip(names, classifiers):
    print ('Fitting {:s} model...'.format(name))
    run_time = %timeit -q -o clf.fit(pipeline.transform(X_train), y_train)

    print ('... predicting')
    y_pred = clf.predict(pipeline.transform(X_train))
    y_pred_test = clf.predict(pipeline.transform(X_test))

    print ('... scoring')
    accuracy  = metrics.accuracy_score(y_train, y_pred)
    precision = metrics.precision_score(y_train, y_pred, average='weighted')
    recall    = metrics.recall_score(y_train, y_pred, average='weighted')

    f1        = metrics.f1_score(y_train, y_pred, average='weighted')
    f1_test   = metrics.f1_score(y_test, y_pred_test, average='weighted')

    # Save the results to dictionaries
    models[name] = clf
    confusion[name] = metrics.confusion_matrix(y_train, y_pred)
    class_report[name] = metrics.classification_report(y_train, y_pred)

    results.append([name, accuracy, precision, recall, f1, f1_test, run_time.best])


results = pd.DataFrame(results, columns=['Classifier', 'Accuracy', 'Precision', 'Recall', 'F1 Train', 'F1 Test', 'Train Time'])
results.set_index('Classifier', inplace= True)

print ('... All done!')
results.sort_values('F1 Train', ascending=False)


In [43]:
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'G:/My Drive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/notebook/results_{timestamp}.xlsx'
df_gridsearch.to_excel(filename, index=False)

In [ ]:
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'G:/MyDrive/Professionele ontwikkeling/Data Science/Explore Data Science Course/Sprint 6_Advanced Classification/Predict/advanced-classification-predict/notebook/dawieloots_predict_gridsearch_{timestamp}.csv'
df_gridsearch.to_csv(filename, index=False)

In [ ]:

print(df_gridsearch)
print(grid_search.get_params)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split

from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Custom transformer to wrap SMOTE

class ResampleAndFeatureSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, k=1000, score_func=chi2, k_neighbors=5):
        self.k = k
        self.score_func = score_func
        self.k_neighbors = k_neighbors
        self.feature_selector = SelectKBest(score_func=self.score_func, k=self.k)
        
    def fit(self, X, y):
        # Print the shape of X_train before preprocessing
               
        # Select features from the resampled data
        self.feature_selector.fit(X, y)
        
        # Print the shape of X_train after preprocessing
        
        return self

    def transform(self, X):
        # Select features from the input data
        X_selected = self.feature_selector.transform(X)
        return X_selected

# Load the 20 Newsgroups dataset
data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
data_subset = {
    'data': data.data[:1000],
    'target': data.target[:1000],
    'target_names': data.target_names
}

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_subset['data'], data_subset['target'], test_size=0.2, random_state=42)

# Define preprocessing steps
preprocessing_steps = [
    # Text preprocessing
    ('vectorizer', TfidfVectorizer()),  # Convert text data into numerical vectors
    ('smote', SMOTE()),
    ('resample_and_feature_select', ResampleAndFeatureSelectTransformer(k=1000, score_func=chi2)),  # Resample and select top k features
    
]

# Define the model
model = KNeighborsClassifier()

# Create the pipeline
pipeline = Pipeline(preprocessing_steps + [('model', model)])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the pipeline
accuracy = pipeline.score(X_test, y_test)
print("Accuracy:", accuracy)

model = pipeline.named_steps['model']

# Obtain the number of features used by the model
attributes = model.__dict__

# Print all attributes
for attr, value in attributes.items():
    print(attr, ":", value)
#n_features = model.n_features_
#print("Number of features used by the model:", n_features)
